## Data Wraggling

In [6]:
import pandas as pd
import numpy as np

from pathlib import Path

In [49]:
DATA_FOLDER = Path("data")
OUTPUT_FOLDER = Path("output")

if not OUTPUT_FOLDER.exists():
    OUTPUT_FOLDER.mkdir()

In [50]:
highspeed_folder = DATA_FOLDER / 'highspeed'

hs_df = pd.read_csv(highspeed_folder / "highspeed.csv")
del hs_df["zip"]


def percent_str_to_percent(s: str) -> float:
    return  int(s[:-1]) / 100

def dollar_amount_to_number(s: str) -> int:
    return int(s.strip()[1:].replace("," ,""))


grade_mapping =  {
        "A+": 5.25,
        "A": 5.0,
        "A-": 4.75,
        "B+": 4.25,
        "B": 4.00,
        "B-": 3.75,
        "C+": 3.25,
        "C": 3.00,
        "C-": 2.75,
        "D+": 2.25,
        "D": 2.00,
        "D-": 1.75,
        "F": 1.00,
    }

hs_df["percentDemocrat"] = hs_df["data__percentDemocrat"].apply(percent_str_to_percent)
hs_df["percentGraduates"] = hs_df["data__percentGraduates"].apply(percent_str_to_percent)
hs_df["percentIndependent"] = hs_df["data__percentIndependent"].apply(percent_str_to_percent)
hs_df["percentNonReligious"] = hs_df["data__percentNonReligious"].apply(percent_str_to_percent)
hs_df["percentReligious"] = hs_df["data__percentReligious"].apply(percent_str_to_percent)
hs_df["percentRepublican"] = hs_df["data__percentRepublican"].apply(percent_str_to_percent)
hs_df["percentSingle"] = hs_df["data__percentSingle"].apply(percent_str_to_percent)

hs_df["avgHomePrice"] = hs_df["data__avgHomePrice"].apply(dollar_amount_to_number)
hs_df["avgCostOfLiving"] = hs_df["data__avgCostOfLiving"].apply(dollar_amount_to_number)


hs_df["gradeAirQuality"] = hs_df["data__gradeAirQuality"].map(grade_mapping)
hs_df["gradeLowTraffic"] = hs_df["data__gradeLowTraffic"].map(grade_mapping)
hs_df["gradeSafety"] = hs_df["data__gradeSafety"].map(grade_mapping)


hs_df = hs_df.loc[:, [col for col in hs_df.columns if not (col.startswith("multipliers__") or col.startswith("data__"))] ]


hs_df.to_csv(OUTPUT_FOLDER / "data.csv")
hs_df

,id,city,state,region,percentDemocrat,percentGraduates,percentIndependent,percentNonReligious,percentReligious,percentRepublican,percentSingle,avgHomePrice,avgCostOfLiving,gradeAirQuality,gradeLowTraffic,gradeSafety
0,1,Palo Alto,CA,West,0.70,0.78,0.02,0.57,0.43,0.28,0.42,2187500,31575,3.25,3.75,4.00
1,2,Naples,FL,South,0.35,0.49,0.01,0.63,0.37,0.65,0.38,308500,19282,3.75,5.25,3.25
2,3,Seattle,WA,West,0.69,0.58,0.02,0.63,0.37,0.29,0.60,579975,28018,5.00,2.25,1.00
3,4,Santa Fe,NM,West,0.74,0.43,0.04,0.51,0.49,0.22,0.60,369000,20840,5.25,5.00,2.25
4,5,Austin,TX,South,0.60,0.45,0.04,0.54,0.46,0.36,0.60,350000,20082,3.00,3.00,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481,482,Victorville,CA,West,0.52,0.12,0.02,0.58,0.42,0.46,0.54,212000,23155,1.00,1.00,2.75
482,483,Perris,CA,West,0.49,0.07,0.02,0.57,0.43,0.49,0.55,276000,20650,1.00,1.00,3.25
483,484,Homestead,FL,South,0.62,0.17,0.01,0.60,0.40,0.38,0.59,202500,23702,4.75,1.00,1.00
484,485,Hesperia,CA,West,0.52,0.10,0.02,0.58,0.42,0.46,0.55,236000,23576,1.00,1.00,3.25


----

In [71]:
from places import fetch_city_coords

coords = hs_df[["city", "state"]].apply(lambda x: fetch_city_coords(*x), axis=1)

In [82]:
coords_df = pd.DataFrame(list(coords), columns=["lat", 'lon'])
coords_df

,lat,lon
0,37.441883,-122.143019
1,26.142036,-81.794810
2,47.606209,-122.332071
3,35.686975,-105.937799
4,30.267153,-97.743061
...,...,...
481,34.536218,-117.292764
482,33.782519,-117.228648
483,25.468722,-80.477557
484,34.426389,-117.300878


In [86]:
data_df = pd.concat([hs_df, coords_df], axis=1)
data_df.to_csv(OUTPUT_FOLDER / "data.csv")
data_df

,id,city,state,region,percentDemocrat,percentGraduates,percentIndependent,percentNonReligious,percentReligious,percentRepublican,percentSingle,avgHomePrice,avgCostOfLiving,gradeAirQuality,gradeLowTraffic,gradeSafety,lat,lon
0,1,Palo Alto,CA,West,0.70,0.78,0.02,0.57,0.43,0.28,0.42,2187500,31575,3.25,3.75,4.00,37.441883,-122.143019
1,2,Naples,FL,South,0.35,0.49,0.01,0.63,0.37,0.65,0.38,308500,19282,3.75,5.25,3.25,26.142036,-81.794810
2,3,Seattle,WA,West,0.69,0.58,0.02,0.63,0.37,0.29,0.60,579975,28018,5.00,2.25,1.00,47.606209,-122.332071
3,4,Santa Fe,NM,West,0.74,0.43,0.04,0.51,0.49,0.22,0.60,369000,20840,5.25,5.00,2.25,35.686975,-105.937799
4,5,Austin,TX,South,0.60,0.45,0.04,0.54,0.46,0.36,0.60,350000,20082,3.00,3.00,2.00,30.267153,-97.743061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481,482,Victorville,CA,West,0.52,0.12,0.02,0.58,0.42,0.46,0.54,212000,23155,1.00,1.00,2.75,34.536218,-117.292764
482,483,Perris,CA,West,0.49,0.07,0.02,0.57,0.43,0.49,0.55,276000,20650,1.00,1.00,3.25,33.782519,-117.228648
483,484,Homestead,FL,South,0.62,0.17,0.01,0.60,0.40,0.38,0.59,202500,23702,4.75,1.00,1.00,25.468722,-80.477557
484,485,Hesperia,CA,West,0.52,0.10,0.02,0.58,0.42,0.46,0.55,236000,23576,1.00,1.00,3.25,34.426389,-117.300878
